# Pipeline

- Data loader: VOC2012
- Network: SSD300
- Loss: MultiboxLoss
- Optimizer
- Tranning, validation

In [9]:
from lib_all import *

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda:0


Train nhanh hơn

In [11]:
torch.backends.cudnn.benchmark = True

# Import 1 số hàm đã tạo

In [12]:
import nbimporter
from data.path import get_path
from data.dataset import MyDataset, collate_fn
from data.processing_data import DataTransform
from data.info_annotation import InfoAnno
from sourc_code.model.model import SSD
from sourc_code.model.multibox_loss import MultiBoxLoss

# I. Dataloader

Tạo path list trainning và validation

In [13]:
root_path ='../../data/data_set/VOCdevkit/VOC2012'
train_images_path, train_annotations_path, val_images_path, val_annotations_path = get_path(root_path)

In [14]:
classes = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

color_mean = (104, 117, 123)
input_size = 300

train_dataset = MyDataset(train_images_path,train_annotations_path, phase = "train", DataTransform = DataTransform(input_size, color_mean), InfoAnno = InfoAnno(classes) )
val_dataset = MyDataset(val_images_path,val_images_path, phase = "val", DataTransform = DataTransform(input_size, color_mean), InfoAnno = InfoAnno(classes) )

# tao data loader
batch_size = 2
train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=collate_fn)

dataloader_dict = {"train": train_dataloader, "val": val_dataloader}


# II. Network

In [15]:
configs = {
    "num_classes": 21,
    "input_size": 300,
    "aspect_num": [4, 6, 6, 6, 4, 4], #so bbox/ pixel  trong so cho cac sorce tu 1 -> 6
    "feature_maps": [38, 19, 10, 5, 3, 1],
    "steps": [8, 16, 32, 64, 100, 300], # size cua pixel
    "min_size": [30, 60, 111, 162, 213, 264],  #size cua default box
    "max_size": [60, 111, 162, 213, 264, 315], #size cua default box
    "aspect_ratio":[[2],[2,3],[2,3],[2,3],[2],[2]]
}

In [16]:
net = SSD(phase = "train", cfg = configs)

load weight

In [17]:
vgg_weigth = torch.load("../../data/download/vgg16_reducedfc.pth")

In [18]:
net.vgg.load_state_dict(vgg_weigth)

<All keys matched successfully>

Hàm số để load thông số khởi tạo cho từng layer

In [19]:
def weights_init(model):
    #neu model la instance nn.Conv2d
    if isinstance(model, nn.Conv2d):
        # sinh ra cac gia tri da duoc chuan hoa de khoi tao model
        nn.init.kaiming_normal_(model.weight.data)
        # neu model co bias
        if model.bias is not None:
            nn.init.constant_(model.bias, 0.0)


Khởi tạo network

In [20]:
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

# III. Khởi tạo loss

In [21]:
criterion = MultiBoxLoss(jaccard_threshold = 0.5, scale_ne_po = 3, device = device)

# IV. Optimizer

In [22]:
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

# V. Trainning

In [23]:
def train_model(net, dataloader_dict, criterion, optimizer, num_epochs):
    net.to(device)
    iteration = 1
    epoch_train_loss = 0.0
    epoch_val_loss = 0.0
    logs = []
    for epoch in range(num_epochs+1):
        # moi epochs mat bao nhieu thoi gian
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print("---"*20)
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("---"*20)

        for phase in ["train", "val"]:
            if phase == "train":
                # goi ham train tu class me (inabale cac tenso de ma luu duoc gradian khi trainning)
                net.train()
                print("(Training)")
            else:
                if (epoch+1) % 10 == 0:
                    net.eval()
                    print("---"*10)
                    print("(Validation)")
                else:
                    continue

            for images, targets in dataloader_dict[phase]:
                # move to GPU
                images = images.to(device)
                targets = [ann.to(device) for ann in targets]
                # init optimizer: dua gradian ve 0 sau moi lan load anh
                optimizer.zero_grad()
                #forward: dua anh vao trong mang
                with torch.set_grad_enabled(phase=="train"): #enabled cac tham so de chua dao ham
                    output = net(images)

                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    if phase == "train":
                        loss.backward() # calculate gradient
                        # gradient co kha nang cao -> cap nhap parameter dan den bat on dinh -> phai cat no di
                        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0) # gia tri parameter khong lon hon 2.0
                        optimizer.step() # update parameters
                        
                        if (iteration % 10) == 0:
                            t_iter_end = time.time()
                            duration = t_iter_end - t_iter_start
                            print("Iteration {} || Loss: {:.4f} || 10iter: {:.4f} sec".format(iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1
                    else:
                        epoch_val_loss += loss.item()
        t_epoch_end = time.time()
        print("---"*20)
        print("Epoch {} || epoch_train_loss: {:.4f} || Epoch_val_loss: {:.4f}".format(epoch+1, epoch_train_loss, epoch_val_loss))
        print("Duration: {:.4f} sec".format(t_epoch_end - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {"epoch": epoch+1, "train_loss": epoch_train_loss, "val_loss": epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("./data/ssd_logs.csv")
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0

        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), "./data/weights/ssd300_" + str(epoch+1) + ".pth")
num_epochs = 100

In [24]:
train_model(net, dataloader_dict, criterion, optimizer, num_epochs=num_epochs)

------------------------------------------------------------
Epoch 1/100
------------------------------------------------------------
(Training)


C:\Users\DINH CHI CONG\OneDrive - Hanoi University of Science and Technology\Study\University\Machine Learning & Data Mining\Object_detection\sourc_code\utils\augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


NameError: name 'outputs' is not defined